In [1]:
import sys
sys.path.append('../../')

In [11]:
import pickle
import math
import pandas as pd
import numpy as np
import tensorflow as tf
import keras.backend as K
from keras.models import load_model
from keras.utils import to_categorical
from utils.dataset import DataSet, LABELS
from utils.metrics import macro_f1
from sklearn.metrics import f1_score, accuracy_score
from env import *

In [3]:
TRAIN_SEQS_PADDED_PKL = os.path.join(CACHES_PATH, 'train_seqs_padded.pkl')
VAL_SEQS_PADDED_PKL = os.path.join(CACHES_PATH, 'val_seqs_padded.pkl')
TEST_SEQS_PADDED_PKL = os.path.join(CACHES_PATH, 'test_seqs_padded.pkl')

In [4]:
raw_dataset = DataSet()
train, val, test = raw_dataset.train, raw_dataset.val, raw_dataset.test

with open(TRAIN_SEQS_PADDED_PKL, 'rb') as f:
    train_seqs_padded = pickle.load(f)
    
with open(VAL_SEQS_PADDED_PKL, 'rb') as f:
    val_seqs_padded = pickle.load(f)
    
with open(TEST_SEQS_PADDED_PKL, 'rb') as f:
    test_seqs_padded = pickle.load(f)
    
train_with_seq = pd.merge(train, train_seqs_padded, on='id')
val_with_seq = pd.merge(val, val_seqs_padded, on='id')
test_with_seq = pd.merge(test, test_seqs_padded, on='id')

In [5]:
model = load_model('../../data/saved_models/linxi/cnn_0', custom_objects={"macro_f1": macro_f1})

In [6]:
val_y = val_with_seq[LABELS[0]] + 2
val_x = np.array(val_with_seq['word_seq'].values.tolist(), dtype = np.int)
y_val_onehot = to_categorical(val_y) 

y_val_prob = model.predict(val_x)

y_val_pred = np.argmax(y_val_prob, axis = 1)

F1_score = f1_score(val_y, y_val_pred, average = 'macro')
F1_score

0.5934300336693034

In [7]:
f1 = macro_f1(tf.constant(y_val_onehot, dtype=tf.float32), tf.constant(y_val_prob, dtype=tf.float32))
sess = tf.Session()
print(sess.run(f1))

0.59343004


In [24]:
model.evaluate(val_x, y_val_onehot)

15000/15000 [==============================] - 1s 69us/step


[0.40891384104092915, 0.8993333333333333, 0.5145913716634115]

In [19]:
acc  =categorical_accuracy(y_val_onehot, y_val_prob)

In [21]:
a = sess.run(acc)

In [22]:
a.mean()

0.89933336